<a href="https://colab.research.google.com/github/MartynaPlomecka/DLM_DigitalFootprint/blob/main/updated__main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 22 07:41:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
## for processing
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for bert
!pip install transformers
!pip install datasets
import transformers
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
import datasets
from datetime import datetime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 4.0 MB 13.9 MB/s 
     |████████████████████████████████| 895 kB 69.1 MB/s 
     |████████████████████████████████| 596 kB 63.5 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
     |████████████████████████████████| 6.6 MB 81.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 13.5 MB/s 
     |████████████████████████████████| 1.1 MB 65.2 MB/s 
     |████████████████████████████████| 136 kB 95.7 MB/s 
     |████████████████████████████████| 212 kB 95.6 MB/s 
     |████████████████████████████████| 127 kB 97.7 MB/s 
     |████████████████████████████████| 144 kB 96.6 MB/

In [4]:
# mount to Google drive
from google.colab import drive
drive.mount('/content/drive')
# dataset paths
dataset_path = '/content/drive/My Drive/embeddings/table4model_clean.csv'
not_labelled_indeed_path = '/content/drive/My Drive/embeddings/dev.txt'



#not labeled data
dtf = pd.read_csv(not_labelled_indeed_path, header = 0)
dtf

Mounted at /content/drive


,text_clean
0,highly skilled individual fantastic energy loo...
1,ambitious professional completed coursework in...
2,experienced project operation manager localiza...
3,highly organized detailoriented professional e...
4,seeking position office environment need varie...
...,...
3812,motivated personable professional experience w...
3813,resultoriented individual reputation strategic...
3814,certified six sigma master black belt 20 year ...
3815,obtain position successful growth oriented com...


# masked language modelling


Masked Language Model (MLM) is the process how BERT was pre-trained. MLM is a powerful pre-training strategy for learning sentence embeddings. This is especially the case when we work on a specialized domain.



In [5]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
from datetime import datetime
no_deprecation_warning=True

In [6]:
model_name = sys.argv[1]
per_device_train_batch_size = 64

save_steps = 1000               #Save model every 1k steps
num_train_epochs = 3            #Number of epochs
use_fp16 = False                #Set to True, if your GPU supports FP16 operations
max_length = 100                #Max length for a text input
do_whole_word_mask = True       #If set to true, whole words are masked
mlm_prob = 0.15                 #Probability that a word is replaced by a [MASK] token

# Load the model
model = AutoModelForMaskedLM.from_pretrained('distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [7]:
transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM

transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM

In [8]:

#output_dir = "output/{}-{}".format(model_name.replace("/", "_"),  datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
output_dir = "output/model"

print("Save checkpoints to:", output_dir)


Save checkpoints to: output/model


In [9]:
##### Load train dataset

train_sentences = []
train_path = '/content/drive/My Drive/embeddings/dev.txt'
with gzip.open(train_path, 'rt', encoding='utf8') if train_path.endswith('.gz') else  open(train_path, 'r', encoding='utf8') as fIn:
    for line in fIn:
        line = line.strip()
        if len(line) >= 10:
            train_sentences.append(line)

print("Train sentences:", len(train_sentences))

dev_sentences = []
if len(sys.argv) >= 4:
    dev_path = sys.argv[3]
    with gzip.open(dev_path, 'rt', encoding='utf8') if dev_path.endswith('.gz') else open(dev_path, 'r', encoding='utf8') as fIn:
        for line in fIn:
            line = line.strip()
            if len(line) >= 10:
                dev_sentences.append(line)

print("Dev sentences:", len(dev_sentences))

#A dataset wrapper, that tokenizes our data on-the-fly
class TokenizedSentencesDataset:
    def __init__(self, sentences, tokenizer, max_length, cache_tokenization=False):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.max_length = max_length
        self.cache_tokenization = cache_tokenization

    def __getitem__(self, item):
        if not self.cache_tokenization:
            return self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)

        if isinstance(self.sentences[item], str):
            self.sentences[item] = self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)
        return self.sentences[item]

    def __len__(self):
        return len(self.sentences)

train_dataset = TokenizedSentencesDataset(train_sentences, tokenizer, max_length)
dev_dataset = TokenizedSentencesDataset(dev_sentences, tokenizer, max_length, cache_tokenization=True) if len(dev_sentences) > 0 else None

##### Training arguments

if do_whole_word_mask:
    data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)
else:
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    evaluation_strategy="steps" if dev_dataset is not None else "no",
    per_device_train_batch_size=per_device_train_batch_size,
    eval_steps=save_steps,
    save_steps=save_steps,
    logging_steps=save_steps,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=use_fp16
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

print("Save tokenizer to:", output_dir)
tokenizer.save_pretrained(output_dir)

trainer.train()

print("Save model to:", output_dir)
model.save_pretrained(output_dir)

print("Training done")


Train sentences: 3818
Dev sentences: 0


tokenizer config file saved in output/model/tokenizer_config.json
Special tokens file saved in output/model/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3818
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 180


Save tokenizer to: output/model


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in output/model/config.json


Save model to: output/model


Model weights saved in output/model/pytorch_model.bin


Training done


In [10]:
data_path = '/content/drive/My Drive/embeddings/dev.txt'

# Model:

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
state_dict = torch.load("output/model/pytorch_model.bin") 
model.load_state_dict(state_dict)

<All keys matched successfully>

## Labeled dataset:

In [12]:
#labeled:

dtf_l = pd.read_csv(dataset_path, header = 0)
labeled = dtf_l[['About', 'grit']]
labeled

,About,grit
0,A passionate Android developer who enjoys buil...,100
1,Experienced Research Fellow with a demonstrate...,75
2,"CPQ Product owner, SAP Workstream Lead and Dig...",100
3,I am a person who has many interests - fashion...,25
4,"Political scientist, specialized in Audiovisua...",75
...,...,...
623,"Aspiring Frontend developer, Homebrewing enthu...",50
624,For many years I have been interested in widel...,0
625,"Skills:- SQL, PostgreSQL - basic level,- Java ...",13
626,"Bilingual Industrial Engineer, with a strong c...",75


In [13]:
# for debugging:
#train = labeled[0:40]
#val = labeled[40:55]
#test= labeled[55:62]

#splits
train = labeled[0:400]
val = labeled[400:550]
test= labeled[550:]

#to json
labeled_json = labeled.to_json()
labeled.to_json('data.json', lines=True, orient='records')

train_json = train.to_json()
train.to_json('data_train.json', lines=True, orient='records')

val_json = val.to_json()
val.to_json('data_val.json', lines=True, orient='records')

test_json = test.to_json()
test.to_json('data_test.json', lines=True, orient='records')

from datasets import Dataset
dataset = Dataset.from_json('data.json')
train_dataset = Dataset.from_json('data_train.json')
val_dataset = Dataset.from_json('data_val.json')
test_dataset = Dataset.from_json('data_test.json')

Using custom data configuration default-d8c5debb388aed6c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-73d086d77cb079ed


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d8c5debb388aed6c/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-027c9def46f2ac21


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-55f38821c24478fc/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-027c9def46f2ac21/0.0.0. Subsequent calls will reuse this data.


In [14]:
#model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, id2label=id2label, label2id=label2id)


## Prepare Datasets
### 1.tokenising the dataset by calling tokenizer. 
### 2.associating the label attribute to each dataset item.


In [15]:
#this is done on labeled dataset in jsons

ds = {"train": train_dataset, "validation": val_dataset, "test": test_dataset}

def preprocess_function(examples):
    label = examples["grit"] 
    examples = tokenizer(examples["About"], truncation=True, padding="max_length", max_length=256)
    examples["label"] = float(label)
    return examples

for split in ds:
    ds[split] = ds[split].filter(lambda e: e['grit'].isnumeric())
    ds[split] = ds[split].map(preprocess_function, remove_columns=["About", "grit"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/397 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/149 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/78 [00:00<?, ?ex/s]

# Metrics
## We can compute metrics to track the model’s improvement during training.
## We retrieve the class with the highest logit (corresponding to the highest probability) for each prediction and compare it with the actual label to calculate the global accuracy score.

In [16]:
import numpy as np
from datasets import load_metric

#metric = load_metric("accuracy")

#def compute_metrics(eval_pred):
#    logits, labels = eval_pred
#    predictions = np.argmax(logits, axis=-1)
#    return metric.compute(predictions=predictions, references=labels)

We put the output directory for the trained model and the learning parameters into TrainingArguments. With load_best_model_at_end and metric_for_best_model, we will keep several best models (i.e. those with the highest accuracy on the validation set) during training and load the best model at the end.

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="../models/camembert-fine-tuned-regression",
    learning_rate=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [18]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    print("1")
    logits, labels = eval_pred
    print("2")
    labels = labels.reshape(-1, 1)
    print(np.array(labels).shape)
    print(np.array(logits).shape)
    logits =[logits[i][0][0] for i in range(len(logits))] 
    mse = mean_squared_error(labels, logits)
    print("2")
    mae = mean_absolute_error(labels, logits)
    print("2")
    r2 = r2_score(labels, logits)
    print("2")
    single_squared_errors = ((logits - labels).flatten()**2).tolist()
    print("2")
    # Compute accuracy 
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
    
    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [19]:

import torch
torch.cuda.empty_cache()
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        #print(outputs[0][:, 0].size())
        #a = torch.nn.Linear(30522, 1).cuda()
        #logits = a(outputs[0][:, 0].cuda())
        logits = outputs[0][0, 0][0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [20]:
torch.cuda.empty_cache()
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics_for_regression,
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 397
  Num Epochs = 64
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 25408
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if sys.path[0] == '':


Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,No log,1174.915894,1174.915894,28.831488,-0.693244,0.000000


***** Running Evaluation *****
  Num examples = 149
  Batch size = 1


1
2
(149, 1)
(149, 256, 30522)
2
2
2
2


Saving model checkpoint to ../models/camembert-fine-tuned-regression/checkpoint-397
Configuration saved in ../models/camembert-fine-tuned-regression/checkpoint-397/config.json
Model weights saved in ../models/camembert-fine-tuned-regression/checkpoint-397/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if sys.path[0] == '':
***** Running Evaluation *****
  Num examples = 149
  Batch size = 1


RuntimeError: ignored

# Evaluation
Note that we rely on the validation set’s accuracy to retrieve the best model. Calling Trainer.evaluate(), we can retrieve the best accuracy attained during training, which is 0.683 (at epoch 16).


In [ ]:
trainer.evaluate()